In [1]:
import numpy  as np
import pandas as pd
import math

## Um den Erwartungswert der interierten Summe eines AR(1) mit dem Wort $W=\underbrace{1\dots1}_{k \text{ mal }}$ zu berechnen, habe ich die Folgende Formel in meiner Masterarbeit in Proposition 7.3.9. gefunden
\begin{align}\mathbf{E}[\langle \mathsf{ISS}(X),\underbrace{1\dots1}_{k \text{ mal }}\rangle]=\begin{cases}0 &  \text{ wenn } k \text{ ungerade }\\(-(\frac{(1-\varphi)\sigma^2}{(1+\varphi)\varphi}))^{\frac{k}{2}}\langle \mathsf{ISS}(Z),\sum_{\omega\in\Omega_k}\mathfrak{fq}_{\omega}\omega\rangle& \text{sonst}\end{cases}\end{align}

### In dieser Formel braucht man $\Omega_k$ die Menge aller Dyck-wörter der Länge $k$ und die dazu gehörigen Koeffizienten, die Freqenzen $\mathfrak{fq}_{\omega}$ in Masterarbeit gennat wurde.
### Die Formel für die Frequenz eines Dyck-Worts ist in der (Bemerkung 7.3.13) wie Folgend gegeben:
### $$\mathfrak{fq}_{\omega}=\mathfrak{fq}_{(f(\mathsf{pos}))}=\prod_{\ell=1}^{\frac{k}{2}}(\mathsf{pos}_{\ell}-1-2(\ell-1))=\prod_{\ell=1}^{\frac{k}{2}}(\mathsf{pos}_{\ell}+   1-2\ell)$$

 ### Der folgende Algorithmus erzeugt die Menge der Dyckwörter mithilfe der Positionen von -1.
 
 ### Dieser Algorithmus ( Funktion ) gibt raus die Dyckwörter $\omega$ und für jedes Dyck-Wort die dazugehörige Frequenz $\mathfrak{fq}_{\omega}$.

In [22]:
def Posintions_of_minus_one(k):
 if k%2==1:
        return('expected value','is zero')
 else:
    #gernrating the dyck words with the length (k)
    k_prime = int(k/2)
    #the array of the positions of the 1's (the "-")
    POS=np.zeros([1,k_prime], dtype = int)
    #the number of the perfect matching of the set {1,2k}, which ist equivalent to each Dyck Word
    Coeff=np.ones([0,1], int)
    A=np.zeros([1,k_prime], dtype = int)
    #the Dyck words represented with "+1","-1"
    Dyck_Words=np.full([0,k], +1)
    #the first( smallest ) word
    for i in range(0,k_prime):
      POS[0][i]=2*(i+1)
      A[0][i]=2*(i+1)
    ind=True
    while ind == True:
        ind=False
        C=np.full([1,k], +1)
        B=np.ones([1,1], dtype = int)
        for i in range(0,k_prime):
            B=B*(A[0][i]+1-2*(i+1)) # the frequency of the Dyck-word
            C[0][A[0][i]-1]=-1 #the Dyck-words
        Coeff=np.concatenate((Coeff,B),axis=0)
        Dyck_Words = np.concatenate((Dyck_Words,C),axis=0)
        for i in range(k_prime-2,-1,-1):
            if A[0][i]< k_prime+1+i:
               A[0][i]=A[0][i]+1
               for j  in range(i+1,k_prime-1):
                   A[0][j]=max(A[0][j-1]+1,2*(j+1))
               ind=True
               POS=np.concatenate((POS,A), axis = 0)
               break
    return(Dyck_Words,Coeff)

### Als 'Sanity Check' kann man überprüfen, ob die Summe des Vektors 'Coeff' gleich die Anzahl der Matchings der Menge [k] ((Vermutung) Bemerkung 7.3.11)  und ob die Anzahl der Einträge im Vektor 'Dyck_Words' ist gleich die die $\frac{k}{2}$-te Catalan-Nummer 

## Die Anzahl der Matchings einer Menge $[k]$ ist gleich $(k-1)!!$ wenn $k$  gerade ist 
### $$(k-1)!!=\frac{k!}{2^{\frac{k}{2}}\frac{k}{2}!}$$
## Die Anzahl der Dyckwörter der Länge $k$ ist gleich  die $\frac{k}{2}$-te Catalan-Nummer
### $$C_{\frac{k}{2}}={\frac {1}{\frac{k}{2}+1}}{k \choose \frac{k}{2}}={\frac {k!}{(\frac{k}{2}+1)!\,\frac{k}{2}!}}$$

In [27]:
k=20

In [28]:
[Dyck_words,frequency_of_Dyckwords]=Posintions_of_minus_one(k)

In [29]:
k_minus_1_double_factorial=math.factorial(k)/(2**int(k/2)*math.factorial(int(k/2))) # the double faculty
catlan_number_k_half=math.factorial(k)/(math.factorial(int(k/2)+1)*math.factorial(int(k/2)))# the (k\2)-th Catalan Number 
print(k_minus_1_double_factorial,sum(frequency_of_Dyckwords))
print(catlan_number_k_half,len(Dyck_words))

654729075.0 [654729075]
16796.0 16796


In [30]:
the_expected_value_of_k_word_of_ones=[]
for i in range(len(Dyck_words)):
    the_expected_value_of_k_word_of_ones.append([frequency_of_Dyckwords[i],Dyck_words[i]])

In [31]:
Df_the_expected_value_of_k_word_of_ones=pd.DataFrame(the_expected_value_of_k_word_of_ones,columns=['frequency','Dyckword'])

### Aus der folgenden Data-Frame erhält man alle Summanden, die in der für den Erwartungswert oben geschiebenen Formel stehen, wobei in jeder Zeile ein Dyck-Wort mit ihrer Frequenz steht.

#### Um die Summe bzw den Erwatungswert zu erhalten, schreibt man diese Summanden mit der dazugehörigen Frequenzen alle als Summe schreiben 

In [38]:
Df_the_expected_value_of_k_word_of_ones

,frequency,Dyckword
0,[1],"[1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, ..."
1,[2],"[1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, ..."
2,[2],"[1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, ..."
3,[4],"[1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, ..."
4,[6],"[1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, ..."
...,...,...
16791,[2177280],"[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1,..."
16792,[2540160],"[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1,..."
16793,[2903040],"[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1,..."
16794,[3265920],"[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1,..."
